# Setup

In [1]:
!pip install pfrl
!pip install textrl
!pip install transformers==4.28.0
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pfrl: filename=pfrl-0.4.0-py3-none-any.whl size=155461 sha256=8afe93eecfcd60bb7932db10f941da20afabfd21fed8b9571b5efd67891c7100
  Stored in directory: /root/.cache/pip/wheels/22/4a/0f/a87cd1ae925086eb3a1b8759f620fcf48e47939fb082946c3b
Successfully built pfrl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Foun

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir ~/.kaggle

!cp /content/drive/MyDrive/OutputsVariousModels/kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download ratthachat/writing-prompts

!unzip writing-prompts.zip

 97% 357M/370M [00:02<00:00, 116MB/s]
100% 370M/370M [00:02<00:00, 154MB/s]
Archive:  writing-prompts.zip
  inflating: writingPrompts/README   
  inflating: writingPrompts/test.wp_source  
  inflating: writingPrompts/test.wp_target  
  inflating: writingPrompts/train.wp_source  
  inflating: writingPrompts/train.wp_target  
  inflating: writingPrompts/valid.wp_source  
  inflating: writingPrompts/valid.wp_target  


In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
import pfrl
from textrl import TextRLEnv, TextRLActor, train_agent_with_evaluation
from transformers import AutoModelForCausalLM, AutoTokenizer
import logging
import sys
from tqdm import tqdm
from random import sample
from torch.utils.data import DataLoader, TensorDataset
from scipy.special import kl_div

logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def get_first_n_lines(file_name_prompts, file_name_stories, n):
  # returns the first n lines from the prompt and story files as concatenated strings
  lines = []
  f1 = open(file_name_prompts, "r")
  f2 = open(file_name_stories, "r")

  while n != 0:
    prompt = f1.readline().replace('<newline>', ' ')
    story = f2.readline().replace('<newline>', ' ')
    if not prompt:
      break

    if prompt != '' and story != '':
      lines.append("Tell me a story about " + prompt + ' ' + story[:-1])
      n = n - 1
  f1.close()
  f2.close()

  return lines

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


I included for testing purposes bad stories that were generated by my previous attempts to be used for training BERT but they can be ignored.

In [6]:
good_story_samples = []
# bad_story_samples = []

# f = open("drive/MyDrive/OutputsVariousModels/chatgpt_datasets_output.txt", "r")
# bad_story_samples = bad_story_samples + f.readlines()
# f.close()

# f = open("drive/MyDrive/OutputsVariousModels/combined_chatgpt_datasets_output.txt", "r")
# bad_story_samples = bad_story_samples + f.readlines()
# f.close()

# f = open("drive/MyDrive/OutputsVariousModels/combined_summary_chatgpt_datasets_output.txt", "r")
# bad_story_samples = bad_story_samples + f.readlines()
# f.close()

# f = open("drive/MyDrive/OutputsVariousModels/summary_chatgpt_datasets_output.txt", "r")
# bad_story_samples = bad_story_samples + f.readlines()
# f.close()

# f = open("drive/MyDrive/OutputsVariousModels/writingprompts_datasets_output.txt", "r")
# bad_story_samples = bad_story_samples + f.readlines()
# f.close()

# # remove final "\n"
# bad_story_samples = [sample[:-1].replace('<newline>', ' ') for sample in bad_story_samples]

# stories from WritingPrompts dataset
good_story_samples = good_story_samples + get_first_n_lines('writingPrompts/train.wp_source', 'writingPrompts/train.wp_target', 24000)

# stories from chatGPT
good_story_samples = good_story_samples + get_first_n_lines('drive/MyDrive/OutputsVariousModels/wp_source-Copy.txt', 'drive/MyDrive/OutputsVariousModels/wp_target-Copy.txt', 12000)

# handpicked prompts that avoid tough requests
f = open("drive/MyDrive/OutputsVariousModels/prompt_samples.txt", "r")
prompt_samples = f.readlines()
f.close()

prompt_samples = [prompt[:-1].replace('<newline>', ' ') for prompt in prompt_samples]

# BERT

In [7]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)
bert_model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [8]:
def trainBERT(model, train_dataset):
  print("BERT Training")
  model.train()

  # Set hyperparameters
  learning_rate = 5e-6
  batch_size = 16
  train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

  optimizer = AdamW(model.parameters(), lr=learning_rate)

  # Training loop
  losses = []
  for batch in tqdm(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, attention_mask, label = batch
    outputs = model(input_ids, attention_mask=attention_mask, labels=label)
    loss = outputs.loss
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  print("BERT Losses" + str(losses))

# GPT2

In [9]:
gpt2_tokenizer = AutoTokenizer.from_pretrained('gpt2')
gpt2_model = AutoModelForCausalLM.from_pretrained('gpt2')
ref_gpt2_model = AutoModelForCausalLM.from_pretrained('gpt2')
gpt2_model.to(device)
ref_gpt2_model.to(device)
ref_gpt2_model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [10]:
def generateStoriesGPT2(model, n=50, rand=True):
  model.eval()
  stories = []
  prompts = prompt_samples[:n]
  if rand:
    prompts = sample(prompt_samples, k=n)

  for i in tqdm(range(len(prompts))):
    prompt = 'Tell me a story about ' + prompts[i]
    input_ids = gpt2_tokenizer.encode(prompt, return_tensors='pt').to(device)

    # Generate text based on the prompt
    generated_text = model.generate(
        input_ids,
        max_length=300,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=gpt2_tokenizer.eos_token_id
    )

    # Decode the generated text back into a string
    stories.append(gpt2_tokenizer.decode(generated_text[0], skip_special_tokens=True).replace("\n", "<newline>"))
  return stories

In [11]:
import random

def trainGPT2(model, tokenizer, reward_model, observaton_list, seed=42):
  print("GPT2 Training")
  reward_model.eval()
  model.train()
  random.seed(seed)
  f = open("train.txt", "w")
  for story in sample(good_story_samples, k=128):
    f.write(story + "\n")
  f.close()

  train_data = TextDataset(
    tokenizer=tokenizer,
    file_path='train.txt',
    block_size=128
  )
  data_collator = DataCollatorForLanguageModeling(
      tokenizer=tokenizer,
      mlm=False
  )
  training_args = TrainingArguments(
      output_dir='./results1',
      overwrite_output_dir=True,
      num_train_epochs=1,
      per_device_train_batch_size=16,
      save_steps=10_000,
      save_total_limit=2,
      gradient_accumulation_steps=4
  )
  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    data_collator=data_collator
  )
  trainer.train()

  random.seed(seed)
  class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish):
      reward = []
      for i in range(len(predicted_list)):
        generated_text = input_item['input'] + ''.join(predicted_list[i]).replace('Ġ', ' ')

        inputs = bert_tokenizer([generated_text], truncation=True, return_tensors="pt")
        inputs.input_ids = inputs.input_ids.to(device)
        inputs.attention_mask = inputs.attention_mask.to(device)
        rewards = reward_model(inputs.input_ids, attention_mask=inputs.attention_mask).logits[0].item()
        if rewards < 0:
          rewards = 0.0
        if rewards > 1:
          rewards = 1.0
        reward.append(rewards-0.5)
      return reward

  env = MyRLEnv(model, tokenizer, observation_input=observaton_list, max_length=200, compare_sample=2)
  actor = TextRLActor(env, model, tokenizer,
                      act_deterministically=False,
                      temperature=1.0,
                      top_k=0,
                      top_p=1.0,)
  agent = actor.agent_ppo(update_interval=2, minibatch_size=16, epochs=2, lr=1e-5)
  scores = []
  for _ in tqdm(range(10)):
    out = train_agent_with_evaluation(
            agent,
            env,
            steps=100,
            eval_n_steps=None,
            eval_n_episodes=1,
            eval_interval=2,
            outdir='bloom—test',
        )
    scores.append(out[1][0]['average_value'])
  print("\nGPT2 scores: " + str(scores))

# GAN

### Train GAN

In [12]:
for gan_iter in tqdm(range(10)):
  observaton_list = []
  labels = []
  print('\nGAN Iteration ' + str(gan_iter))

  # add bad stories
  observaton_list += generateStoriesGPT2(gpt2_model, 50)
  labels += [1 for _ in range(50)]

  #observaton_list += sample(bad_story_samples, k=50)
  #labels += [1 for _ in range(50)]

  # add good stories
  observaton_list += sample(good_story_samples, k=50)
  labels += [0 for _ in range(50)]

  # created dataloader for BERT model
  inputs = bert_tokenizer(observaton_list, padding=True, truncation=True, return_tensors="pt")
  train_dataset = TensorDataset(inputs.input_ids, inputs.attention_mask, torch.FloatTensor(labels))
  print('BERT training set completed')

  trainBERT(bert_model, train_dataset)

  prompt_list = []
  for prompt in sample(prompt_samples, k=100):
    prompt_list.append({"input" : prompt[:-1]})

  trainGPT2(gpt2_model, gpt2_tokenizer, bert_model, prompt_list, seed=gan_iter)


  0%|          | 0/10 [00:00<?, ?it/s]


GAN Iteration 0



100%|██████████| 50/50 [03:27<00:00,  4.15s/it]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BERT training set completed
BERT Training



100%|██████████| 7/7 [00:09<00:00,  1.37s/it]
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


...


100%|██████████| 7/7 [00:10<00:00,  1.43s/it]


BERT Losses[0.10370840132236481, 0.04642891511321068, 0.09967368841171265, 0.16655871272087097, 0.10655301064252853, 0.04209061339497566, 0.06849709153175354]
GPT2 Training


Step,Training Loss



100%|██████████| 10/10 [1:25:37<00:00, 513.75s/it]


GPT2 scores: [-1.6880195, -1.6880051, -1.6979202, -1.6810266, -1.6839569, -1.6747447, -1.6743175, -1.685586, -1.6823516, -1.6734979]


In [13]:
gpt2_model.save_pretrained("drive/MyDrive/OutputsVariousModels/GPT2_Generator")
bert_model.save_pretrained("drive/MyDrive/OutputsVariousModels/BERT_Discriminator")

### KL divergence
Tests how different the output of the final GPT-2 model is compared to the initial pretrained one.

In [14]:
kl_scores = []
gpt2_model.eval()
ref_gpt2_model.eval()
for prompt in sample(prompt_samples, k=10):
  input_ids = gpt2_tokenizer.encode(prompt, return_tensors='pt').to(device)

  probs = torch.softmax(gpt2_model(input_ids).logits[0, -1, :], 0).tolist()
  ref_probs = torch.softmax(ref_gpt2_model(input_ids).logits[0, -1, :], 0).tolist()

  kl_scores.append(sum(kl_div(probs, ref_probs)))

print(kl_scores)
print(sum(kl_scores)/len(kl_scores))

[0.8075677413873651, 2.4022767942508056, 1.1950949372664483, 1.5265947580674686, 1.436742840176895, 3.7097544667417797, 2.4048529222663952, 1.01148877333619, 4.394669147117747, 1.1145582559076102]
2.0003600636518706


### Generate short stories for analysis

In [15]:
f = open("drive/MyDrive/OutputsVariousModels/GAN_vanilla.txt", "w")
for story in generateStoriesGPT2(gpt2_model, 200, rand=False):
  f.write(story + '\n')
f.close()

100%|██████████| 200/200 [15:46<00:00,  4.73s/it]


### Check adversarial behavior

In [16]:
bert_model.eval()
gpt2_model.eval()
for prompt in prompt_samples[:5]:
  input_ids = gpt2_tokenizer.encode(prompt, return_tensors='pt').to(device)

  # Generate text based on the prompt
  generated_text = gpt2_model.generate(
      input_ids,
      max_length=300,
      num_beams=5,
      no_repeat_ngram_size=2,
      early_stopping=True,
      pad_token_id=gpt2_tokenizer.eos_token_id,
  )

  # Decode the generated text back into a string
  generated_text = gpt2_tokenizer.decode(generated_text[0], skip_special_tokens=True).replace("\n", "<newline>")
  inputs = bert_tokenizer([generated_text], truncation=True, return_tensors="pt")
  inputs.input_ids = inputs.input_ids.to(device)
  inputs.attention_mask = inputs.attention_mask.to(device)
  print(prompt)
  print(generated_text)
  print(bert_model(inputs.input_ids, attention_mask=inputs.attention_mask).logits[0])
  print()

[ WP ] You are an assassin sent from the future to kill women that would give birth to evil men and women , for the greater good . In the future , you are known as The Purifier . In the present , you are known by a very different name : Jack the Ripper .
[ WP ] You are an assassin sent from the future to kill women that would give birth to evil men and women, for the greater good. In the future, you are known as The Purifier. In the present, you are known by a very different name : Jack the Ripper.<newline> “ What ’ s the meaning of the word? ” ‘ I don't know. I mean, I know it sounds crazy, but what does that even mean? It means that you're going to do whatever it takes to make sure that no one ever has an abortion, that there is no child without a mother and that all children are created according to the will of God. It's not crazy at all. What it does mean is that if you do what you say, there will be a baby born, and then you will kill that baby and give it to someone else. That pe

In [ ]:
import time
time.sleep(120)

from google.colab import runtime
runtime.unassign()